In [123]:
import numpy as np
import requests
import pandas as pd
from IPython.display import display, HTML

In [73]:
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve
import http.client as httplib 
from astropy.table import Table

In [68]:
tce_csv = "./data/tce_stats/tess2018206190142-s0001-s0009_dvr-tcestats.csv"

In [69]:
tce_data = pd.read_csv(tce_csv, skiprows=6)

In [42]:
tce_data.head(50)

tceid      ticid  planetNumber  planetCandidateCount  \
0   00101363142-01  101363142             1                     1   
1   00101364075-01  101364075             1                     1   
2   00101365484-01  101365484             1                     1   
3   00101395259-01  101395259             1                     1   
4   00101395750-01  101395750             1                     3   
5   00101395750-02  101395750             2                     3   
6   00101395750-03  101395750             3                     3   
7   00101417386-01  101417386             1                     3   
8   00101417386-02  101417386             2                     3   
9   00101417386-03  101417386             3                     3   
10  00117739806-01  117739806             1                     2   
11  00117739806-02  117739806             2                     2   
12  00118170779-01  118170779             1                     1   
13  00118307280-01  118307280             1                     2   
14  00118307280-02  118307280             2                     2   
15  00118313102-01  118313102             1                     5   
16  00118313102-02  118313102             2                     5   
17  00118313102-03  118313102             3                     5   
18  00118313102-04  118313102             4                     5   
19  00118313102-05  118313102             5                     5   
20  00118316147-01  118316147             1                     1   
21  00118414770-01  118414770             1                     2   
22  00118414770-02  118414770             2                     2   
23  00118417121-01  118417121             1                     1   
24  00118541883-01  118541883             1                     3   
25  00118541883-02  118541883             2                     3   
26  00118541883-03  118541883             3                     3   
27  00119292328-01  119292328             1                     1   
28  00119327278-01  119327278             1                     3   
29  00119327278-02  119327278             2                     3   
30  00119327278-03  119327278             3                     3   
31  00122330743-01  122330743             1                     2   
32  00122330743-02  122330743             2                     2   
33  00122907209-01  122907209             1                     3   
34  00122907209-02  122907209             2                     3   
35  00122907209-03  122907209             3                     3   
36  00123029030-01  123029030             1                     2   
37  00123029030-02  123029030             2                     2   
38  00123036723-01  123036723             1                     1   
39  00123153249-01  123153249             1                     1   
40  00123380233-01  123380233             1                     1   
41  00123382400-01  123382400             1                     2   
42  00123382400-02  123382400             2                     2   
43  00123480394-01  123480394             1                     1   
44  00123482865-01  123482865             1                     1   
45  00123699904-01  123699904             1                     1   
46  00123702439-01  123702439             1                     1   
47  00123744995-01  123744995             1                     3   
48  00123744995-02  123744995             2                     3   
49  00123744995-03  123744995             3                     3   

    starTeffKelvin  starLoggCgs  orbitalPeriodDays  orbitalPeriodDays_err  \
0           3237.0      4.99853          28.576300               0.002830   
1           5758.0      4.40457          24.406100               0.016645   
2           3536.0      4.43800          29.930900               0.004922   
3           6422.0      4.20608           7.574500               0.000261   
4           8596.0      3.66834           0.247308               0.000015   
5           8596.0      3.66834

In [35]:
len(tce_data)

3889

In [36]:
tessids = []
for tessid in tce_data['ticid']:
    if tessid not in tessids:
        tessids.append(tessid)

In [119]:
tceids = []
for tce in tce_data['tceid']:
    if tce not in tceids:
        tceids.append(tce)

In [37]:
# I think the missing columns can ge gotten from astroquery relatively easily
tce_data.columns

Index(['tceid', 'ticid', 'planetNumber', 'planetCandidateCount',
       'starTeffKelvin', 'starLoggCgs', 'orbitalPeriodDays',
       'orbitalPeriodDays_err', 'planetRadiusEarthRadii',
       'planetRadiusEarthRadii_err', 'ratioPlanetRadiusToStarRadius',
       'ratioSemiMajorAxisToStarRadius', 'semiMajorAxisAu',
       'starRadiusSolarRadii', 'transitDepthPpm', 'transitDepthPpm_err',
       'transitDurationHours', 'transitEpochBtjd', 'transitEpochBtjd_err',
       'transitIngressTimeHours', 'minImpactParameter', 'ws_mes',
       'ws_mesphase', 'expectedtransitcount', 'chiSquare2', 'mes', 'maxses',
       'equilibriumTempKelvin', 'InsolationFlux'],
      dtype='object')

In [54]:
tce_data.loc[tce_data['ticid'] == 101395750]['starRadiusSolarRadii'].values[0]

3.5552900000000003

In [45]:
tessids[10]

118316147

In [74]:
def mastQuery(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content

In [75]:
# Astroquery request function

def aqTicReq(tessid, column):
    requestTic = {"service":"Mast.Catalogs.Filtered.Tic", 
                  "format":"json",
                  "params":{
                      "columns":"c.{}".format(str(column)), #Column name to get
                      "filters":[
                           {"paramName":"ID",   #Filter by ID (tessid/ticid)
                             "values":[str(tessid)]}]
                        }
    }

    headers,outString = mastQuery(requestTic)
    outData = json.loads(outString)      #This isn't entirely necessary here
    return(outData['data'][0][str(column)])

In [124]:
# Planet name assembler

def nameResolver(ticid):
    request = {'service':'Mast.Name.Lookup',
                'params':{'input':str(ticid),
                          'format':'json'},
    }
     
    headers,outString = mastQuery(request)
     
    outData = json.loads(outString)
    
    return outData

def pnameGenerator(tceid):
    tessid, pl_number = tceid.split("-")
    tessid = str(int(tessid))
    pl_dict = {"01":"a", "02":"b", "03":"c", "04":"d", 
               "05":"e", "06":"f", "07":"g", "08":"h",
               "09":"i", "10":"j", "11":"k", "12":"l"}
    star_name = nameResolver(tessid)['resolvedCoordinate'][0]['canonicalName']
    return(str(star_name + "-" + pl_dict[pl_number]))


In [130]:
# exoMAST info getters: urls

planeturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/"
dvurl = "https://exo.mast.stsci.edu/api/v0.1/dvdata/tess/"
header_secinfo={}

In [144]:
# exoMAST info getters: sector getter

def getSector(tessid):
    name = nameResolver(tessid)['resolvedCoordinate'][0]['canonicalName']
    url = planeturl + "/identifiers/"
    params = {"name":name}
    r = requests.get(url=url, params=params, headers=header_secinfo)
    planet_names = r.json()
    tce = planet_names['tessTCE']
    url = dvurl + str(123482865) + '/tces/'
    myparams = {"tce" : tce}

    r = requests.get(url = url, params = myparams, headers = header)
    sectorInfo = r.json()

    sectors = [x[:11] for x in sectorInfo["TCE"] if tce in x]
    return(sectors[0][:5])

In [145]:
# test cell
getSector(123482865)

's0008'

In [ ]:
# exoMAST info getters: constellation getter

def getConstellation(tceid):
    

In [ ]:
# test cell

In [ ]:
# exoMAST info getters: planet mass getter

def getPMass(tceid):
    tessid, pl_number = tceid.split("-")
    star_name = nameResolver(tessid)['resolvedCoordinate'][0]['canonicalName']
    url = planeturl + "/identifiers/"
    params = {"name":star_name}
    r = requests.get(url=url, params=params, headers=header_secinfo)
    planet_names = r.json()
    tce = planet_names['tessTCE']
    url = dvurl + str(123482865) + '/tces/'
    myparams = {"tce" : tce}

    r = requests.get(url = url, params = myparams, headers = header)
    sectorInfo = r.json()

    sectors = [x[:11] for x in sectorInfo["TCE"] if tce in x]
    return(sectors[0][:5])
    

In [ ]:
# test cell

In [122]:
radec = [aqTicReq(101417386, 'ra'), aqTicReq(101417386, 'dec')]
radec

[148.931441200614, -19.6770836403199]

In [ ]:
# Using the same format as the dummy data generator.
# Randoms will be replaced with indexing from various sources.
planetids = [pln['plid'][randint(1,4002)] for x in range(rows)]
for tessid in tessids:
    Stars = pd.DataFrame({
        "star_tessid": int(tessid),
        "twomass": aqTicReq(tessid, 'TWOMASS')
        "gaiaid": aqTicReq(tessid, 'GAIA')
        "sector": getSector(tessid)
        "magnitude": aqTicReq(tessid, 'Tmag')
        "luminosity": aqTicReq(tessid, 'lumclass')
        "star_mass": aqTicReq(tessid, 'mass')
        "star_radius": tce_data.loc[tce_data['ticid'] == tessid]['starRadiusSolarRadii'].values[0]
        "constellation" : #choice(["Andromeda", "Aquila", 
                               #"Canis Minor","Cetus",
                               #"Corvus","Leo Minor",
                               #"Orion","Eridanus"]),
        "disposition": aqTicReq(tessid, 'disposition'),
        "rightascension": str(round(float(aqTicReq(tessid, 'ra')), 8)),
        "declination": str(round(float(aqTicReq(118313102, 'ra')), 8)),
        })

    
for tce in tceids:
    Planets = pd.DataFrame({
        "planetid": pnameGenerator(tce),
        "star_tessid": tce_data.loc[tce_data['tceid'] == tce]['ticid'],
        "orbit_period": tce_data.loc[tce_data['tceid'] == tce]['orbitPeriodDays'],
        "planet_mass": 
        "planet_radius": tce_data.loc[tce_data['tceid'] == tce]['planetRadiusEarthRadii'],
    })
    
#**Still Need:
# constellation
# planet mass


In [79]:
tce_data.columns

Index(['tceid', 'ticid', 'planetNumber', 'planetCandidateCount',
       'starTeffKelvin', 'starLoggCgs', 'orbitalPeriodDays',
       'orbitalPeriodDays_err', 'planetRadiusEarthRadii',
       'planetRadiusEarthRadii_err', 'ratioPlanetRadiusToStarRadius',
       'ratioSemiMajorAxisToStarRadius', 'semiMajorAxisAu',
       'starRadiusSolarRadii', 'transitDepthPpm', 'transitDepthPpm_err',
       'transitDurationHours', 'transitEpochBtjd', 'transitEpochBtjd_err',
       'transitIngressTimeHours', 'minImpactParameter', 'ws_mes',
       'ws_mesphase', 'expectedtransitcount', 'chiSquare2', 'mes', 'maxses',
       'equilibriumTempKelvin', 'InsolationFlux'],
      dtype='object')

In [110]:
tce_data.loc[tce_data['ticid'] == 101395750]['tceid'].values[0].split()

str

In [ ]:
for tessid in tessids[:100]:
    name = nameResolver(tessid)['resolvedCoordinate'][0]['canonicalName']
    url = planeturl + "/identifiers/"
    params = {"name":name}
    r = requests.get(url=url, params=params, headers=header_secinfo)
    planet_names = r.json()
    tce = planet_names['tessTCE']
    url = dvurl + str(123482865) + '/tces/'
    myparams = {"tce" : tce}

    r = requests.get(url = url, params = myparams, headers = header)
    if planet_names['planetNames']